# 雙動能交易策略回測專案

本筆記本包含完整的回測邏輯、參數最佳化、績效計算及報表生成。

In [ ]:
import pandas as pd
import numpy as np
import itertools
import json
import matplotlib.pyplot as plt
import xlsxwriter
from datetime import datetime

## 1. 資料讀取與處理

In [ ]:
def load_data(filepath):
    df = pd.read_excel(filepath, header=[0, 1])
    asset_names = {col[0]: col[1] for col in df.columns if 'Unnamed' not in col[0]}
    df.columns = [col[0] if 'Unnamed' not in col[0] else 'Date' for col in df.columns]
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    df = df.apply(pd.to_numeric, errors='coerce')
    return df, asset_names

df, asset_names = load_data('16ETF-V1.xlsx')

## 2. 參數最佳化結果

In [ ]:
best_params = {
    "0050": {
        "params": [
            2,
            3,
            7
        ],
        "calmar": 1.433808796039839
    },
    "0056": {
        "params": [
            7,
            8,
            29
        ],
        "calmar": 1.6160476311303442
    },
    "00631L": {
        "params": [
            1,
            6,
            7
        ],
        "calmar": 1.8240533776341112
    },
    "006208": {
        "params": [
            1,
            3,
            8
        ],
        "calmar": 1.6017116253982162
    },
    "00878": {
        "params": [
            5,
            16,
            73
        ],
        "calmar": 1.7159709355301611
    },
    "00919": {
        "params": [
            1,
            2
        ],
        "calmar": 14.605079771246636
    },
    "00891": {
        "params": [
            1,
            6,
            17
        ],
        "calmar": 3.048397955056761
    },
    "00713": {
        "params": [
            1,
            3,
            4
        ],
        "calmar": 2.5267952089798213
    },
    "00715L": {
        "params": [
            3,
            18,
            78
        ],
        "calmar": 1.1504685998134014
    },
    "00642U": {
        "params": [
            13,
            24,
            56
        ],
        "calmar": 1.03234263835753
    },
    "00635U": {
        "params": [
            9,
            10
        ],
        "calmar": 0.9013456149123213
    },
    "00708L": {
        "params": [
            1,
            26,
            29
        ],
        "calmar": 0.5376342848129401
    },
    "00680L": {
        "params": [
            1,
            2,
            10
        ],
        "calmar": 0.4693240213970293
    },
    "00679B": {
        "params": [
            1,
            10,
            15
        ],
        "calmar": 0.5461066822574183
    },
    "00751B": {
        "params": [
            1,
            3,
            27
        ],
        "calmar": 0.9210437155473732
    },
    "00688L": {
        "params": [
            1,
            5,
            10
        ],
        "calmar": 0.49108244546224356
    }
}

## 3. 策略回測核心邏輯

In [ ]:
def run_backtest(df, best_params):
    dates = df.index
    n = len(dates)
    mom_df = pd.DataFrame(index=dates)
    for asset in df.columns:
        p = best_params[asset]['params']
        prices = df[asset]
        if len(p) == 2:
            mom = (prices/prices.shift(p[0]) + prices/prices.shift(p[1]) - 2)/2
        else:
            mom = (prices/prices.shift(p[0]) + prices/prices.shift(p[1]) + prices/prices.shift(p[2]) - 3)/3
        mom_df[asset] = mom
        
    cash = 10000000.0
    holdings = {}
    history = []
    trade_logs = []
    rebalance_indices = np.arange(0, n, 5)
    pending_update = None

    for i in range(n):
        if i >= 2 and (i-2) in rebalance_indices:
            if pending_update:
                holdings, cash, t_info = pending_update
                trade_logs.append(t_info)
                pending_update = None
        val = cash + sum(shares * df.iloc[i][asset] for asset, shares in holdings.items())
        history.append({'Date': dates[i], 'Cash': cash, 'Equity': val, 'Holdings': holdings.copy()})
        if i in rebalance_indices and i + 1 < n:
            top_assets = mom_df.iloc[i][mom_df.iloc[i]>0].sort_values(ascending=False).head(2).index.tolist()
            trade_prices = df.iloc[i+1]
            to_keep = [a for a in holdings if a in top_assets]
            to_sell = [a for a in holdings if a not in top_assets]
            to_buy = [a for a in top_assets if a not in holdings]
            new_cash = cash
            for a in to_sell: new_cash += holdings[a] * trade_prices[a]
            new_holdings = {a: holdings[a] for a in to_keep}
            if len(to_buy) > 0:
                cpb = new_cash / len(to_buy)
                for a in to_buy: new_holdings[a] = cpb / trade_prices[a]
                new_cash = 0
            pending_update = (new_holdings, new_cash, {'SignalDate': dates[i], 'TradeDate': dates[i+1], 'TopAssets': top_assets})
    return pd.DataFrame(history), trade_logs
history_df, trade_logs = run_backtest(df, best_params)

## 4. 績效圖表

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(history_df['Date'], history_df['Equity'])
plt.title('Equity Curve')
plt.grid(True)
plt.show()